In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [9]
commitnumber = de5d43e
molecule = {'ch4': 1.8e-06}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
  dv = 0.001
  klin = 2e-21
  molecule = ch4
  ng_adju = [0, 0]
  ng_refs = [2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.7, 0.7), (0.4, 0.6, 0.75)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.048886                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.008062
0.000750      2.0 -0.044636                  2.0 -0.005492
0.001052      3.0 -0.035353                  3.0  0.000575
0.001476      4.0 -0.025072                  4.0  0.007148
0.002070      5.0 -0.013849                  5.0  0.014372
0.002904      6.0 -0.001629                  6.0  0.022317
0.004074      7.0  0.011642                  7.0  0.031020
0.005714      8.0  0.026034                  8.0  0.040539
0.008015      9.0  0.041584                  9.0  0.050940
0.011243     10.0  0.058361                 10.0  0.062305
0.015771     11.0  0.076396                 11.0  0.074688
0.022122     12.0  0.095713                 12.0  0.088158
0.031031     13.0  0.116256                 13.0  0.102723
0.043528     14.0  0.135205                 14.0  0.116588
0.061057     15.0  0.149673                 15.0  0.127914
0.085645     16.0  0.161735                 16.0  0.138109
0.120136     17.0  0.173516                 17.0  0.148697
0.168516     18.0  0.185014                 18.0  0.159688
0.236378     19.0  0.196248                 19.0  0.171066
0.331549     20.0  0.207499                 20.0  0.182797
0.465100     21.0  0.217729                 21.0  0.193552
0.652400     22.0  0.198993                 22.0  0.183149
0.915100     23.0  0.146774                 23.0  0.147014
1.283650     24.0  0.096346                 24.0  0.108449
1.800600     25.0  0.059257                 25.0  0.075781
2.525700     26.0  0.033524                 26.0  0.048680
3.542800     27.0  0.016602                 27.0  0.027047
4.969550     28.0  0.008060                 28.0  0.012730
6.970850     29.0  0.005762                 29.0  0.005909
9.778100     30.0  0.005070                 30.0  0.003130
13.715850    31.0  0.004063                 31.0  0.001977
19.239350    32.0  0.002658                 32.0  0.001454
26.987250    33.0  0.002382                 33.0  0.002267
37.855300    34.0  0.003332                 34.0  0.003990
53.100050    35.0  0.003867                 35.0  0.004987
73.887500    36.0  0.003960                 36.0  0.005353
97.662500    37.0  0.003649                 37.0  0.005138
121.437500   38.0  0.002439                 38.0  0.003917
145.212500   39.0  0.000708                 39.0  0.002018
168.987500   40.0 -0.000969                 40.0 -0.000033
192.762500   41.0 -0.002721                 41.0 -0.002303
216.537500   42.0 -0.004641                 42.0 -0.004862
240.312500   43.0 -0.006852                 43.0 -0.007748
264.087500   44.0 -0.009578                 44.0 -0.011061
287.862500   45.0 -0.012887                 45.0 -0.014600
311.637500   46.0 -0.014531                 46.0 -0.016345
335.412500   47.0 -0.013063                 47.0 -0.015139
359.187500   48.0 -0.010524                 48.0 -0.012694
382.962500   49.0 -0.008206                 49.0 -0.010228
406.737500   50.0 -0.005959                 50.0 -0.007782
430.512500   51.0 -0.003671                 51.0 -0.005291
454.287500   52.0 -0.001290                 52.0 -0.002739
478.062500   53.0  0.001207                 53.0 -0.000094
501.837500   54.0  0.003822                 54.0  0.002625
525.612500   55.0  0.006543                 55.0  0.005427
549.387500   56.0  0.009336                 56.0  0.008280
573.162500   57.0  0.012153                 57.0  0.011141
596.937500   58.0  0.014934                 58.0  0.013949
620.712500   59.0  0.017616                 59.0  0.016635
644.487500   60.0  0.020150                 60.0  0.019195
668.262500   61.0  0.022510                 61.0  0.021587
692.037500   62.0  0.024702                 62.0  0.023851
715.812500   63.0  0.026765                 63.0  0.025995
739.587500   64.0  0.028762                 64.0  0.028083
76

# Fluxes by Layer

CRD                        CLIRAD                \
                        flug      fldg     fnetg      flug          fldg   
pressure     level                                                         
0.000000e+00 1     -8.185270  0.000000 -8.185270       NaN           NaN   
1.000000e-08 1           NaN       NaN       NaN -8.177808  9.420030e-08   
6.244000e-04 2     -8.185276  0.000003 -8.185273 -8.177811  1.948627e-06   
8.759000e-04 3     -8.185279  0.000004 -8.185275 -8.177812  2.741582e-06   
1.228600e-03 4     -8.185282  0.000006 -8.185276 -8.177813  3.982756e-06   
1.723400e-03 5     -8.185287  0.000009 -8.185278 -8.177814  5.916094e-06   
2.417400e-03 6     -8.185292  0.000013 -8.185279 -8.177816  8.924982e-06   
3.390900e-03 7     -8.185299  0.000020 -8.185279 -8.177818  1.360531e-05   
4.756500e-03 8     -8.185308  0.000031 -8.185277 -8.177821  2.087669e-05   
6.672000e-03 9     -8.185319  0.000048 -8.185271 -8.177823  3.215892e-05   
9.358900e-03 10    -8.185332  0.000074 -8.185258 -8.177824  4.964413e-05   
1.312780e-02 11    -8.185346  0.000115 -8.185232 -8.177823  7.671253e-05   
1.841450e-02 12    -8.185360  0.000176 -8.185184 -8.177818  1.185604e-04   
2.583020e-02 13    -8.185371  0.000271 -8.185100 -8.177805  1.831628e-04   
3.623230e-02 14    -8.185373  0.000416 -8.184957 -8.177778  2.826949e-04   
5.082340e-02 15    -8.185358  0.000635 -8.184723 -8.177728  4.341155e-04   
7.129060e-02 16    -8.185316  0.000955 -8.184360 -8.177644  6.598072e-04   
1.000000e-01 17    -8.185230  0.001420 -8.183810 -8.177507  9.927455e-04   
1.402710e-01 18    -8.185072  0.002090 -8.182982 -8.177288  1.482909e-03   
1.967600e-01 19    -8.184791  0.003046 -8.181744 -8.176939  2.202741e-03   
2.759970e-01 20    -8.184304  0.004402 -8.179902 -8.176387  3.256253e-03   
3.871000e-01 21    -8.183473  0.006301 -8.177172 -8.175515  4.790345e-03   
5.431000e-01 22    -8.182064  0.008916 -8.173148 -8.174148  7.001168e-03   
7.617000e-01 23    -8.180073  0.012078 -8.167995 -8.172285  9.880826e-03   
1.068500e+00 24    -8.177860  0.015199 -8.162661 -8.170184  1.312311e-02   
1.498800e+00 25    -8.175639  0.017889 -8.157750 -8.168007  1.647446e-02   
2.102400e+00 26    -8.173483  0.019970 -8.153513 -8.165828  1.971430e-02   
2.949000e+00 27    -8.171500  0.021349 -8.150151 -8.163761  2.253023e-02   
4.136600e+00 28    -8.169886  0.022070 -8.147816 -8.161990  2.456472e-02   
5.802500e+00 29    -8.168746  0.022521 -8.146225 -8.160611  2.569791e-02   
8.139200e+00 30    -8.167971  0.023340 -8.144630 -8.159537  2.625997e-02   
1.141700e+01 31    -8.167599  0.024937 -8.142662 -8.158829  2.676764e-02   
1.601470e+01 32    -8.168038  0.027590 -8.140449 -8.158921  2.793585e-02   
2.246400e+01 33    -8.170210  0.031792 -8.138418 -8.160706  3.083259e-02   
3.151050e+01 34    -8.174867  0.039002 -8.135865 -8.164899  3.745552e-02   
4.420010e+01 35    -8.182267  0.051410 -8.130857 -8.171756  5.031101e-02   
6.200000e+01 36    -8.193547  0.070844 -8.122704 -8.182449  7.152132e-02   
8.577500e+01 37    -8.209869  0.098317 -8.111551 -8.198238  1.023884e-01   
1.095500e+02 38    -8.227569  0.126294 -8.101274 -8.215749  1.343712e-01   
1.333250e+02 39    -8.247599  0.153194 -8.094404 -8.236085  1.657394e-01   
1.571000e+02 40    -8.271002  0.178591 -8.092411 -8.260336  1.956734e-01   
1.808750e+02 41    -8.298090  0.202949 -8.095140 -8.288915  2.241609e-01   
2.046500e+02 42    -8.329257  0.226454 -8.102804 -8.322368  2.511254e-01   
2.284250e+02 43    -8.365085  0.249211 -8.115874 -8.361461  2.765239e-01   
2.522000e+02 44    -8.406472  0.271301 -8.135172 -8.407126  3.003662e-01   
2.759750e+02 45    -8.454950  0.292803 -8.162147 -8.460655  3.227408e-01   
2.997500e+02 46    -8.512747  0.314307 -8.198440 -8.523241  3.442049e-01   
3.235250e+02 47    -8.578897  0.339532 -8.239365 -8.593033  3.679592e-01   
3.473000e+02 48    -8.649009  0.372855 -8.276154 -8.665950  3.982355e-01   
3.710750e+02 49    -8.720446  0.414654 -8.305793 -8.739682  4.362147e-01 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')